# Сравнение поисковых деревьев
### Выполнили Вихрев И., Репин В. гр 382006-3м

In [25]:
import sys
from pathlib import Path
import time
import timeit
import pandas as pd
sys.path.append(str(Path.cwd().resolve()/'python_bindings'))
import trees
st_time = []
avl_time = []
treap_time = []
rbtree_time = []
sortarr_time = []

In [26]:
dir(trees)

['AVLTreeInt',
 'AVLTreeString',
 'RBTreeInt',
 'RBTreeString',
 'SortedArrInt',
 'SortedArrString',
 'SplayTreeInt',
 'SplayTreeString',
 'TreapInt',
 'TreapString',
 '__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__test__']

In [27]:
import random
unique_num_set = random.sample(range(1, 10**8), 10**6)

In [28]:
st = trees.SplayTreeInt()

In [29]:
for i in unique_num_set:
    start = time.time_ns()
    st.insert(i)
    end = time.time_ns()
    st_time.append(end-start)

In [30]:
avl = trees.AVLTreeInt()

In [31]:
for i in unique_num_set:
    start = time.time_ns()
    avl.insert(i)
    end = time.time_ns()
    avl_time.append(end-start)

In [32]:
treap = trees.TreapInt()
for i in unique_num_set:
    start = time.time_ns()
    treap.insert(i)
    end = time.time_ns()
    treap_time.append(end-start)

In [33]:
rbtree = trees.RBTreeInt()
for i in unique_num_set:
    start = time.time_ns()
    rbtree.insert(i)
    end = time.time_ns()
    rbtree_time.append(end-start)

In [34]:
sortarr = trees.SortedArrInt()
for i in unique_num_set:
    start = time.time_ns()
    sortarr.insert(i)
    end = time.time_ns()
    sortarr_time.append(end-start)

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style = 'whitegrid', rc={'figure.figsize':(20, 10)})

## splay tree

In [ ]:
num = range(0, len(st_time))
plt.scatter(num, st_time)

## avl tree

In [ ]:
plt.scatter(num, avl_time)

## treap

In [ ]:
plt.scatter(num, treap_time)

## red-black tree

In [ ]:
plt.scatter(num, rbtree_time)

## sorted arr

In [ ]:
plt.scatter(num, sortarr_time)